In [1]:
import pandas as pd
import numpy as np


from os import listdir
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras_tqdm import TQDMNotebookCallback


Using TensorFlow backend.


In [4]:
%cd ~/dlcookbook/dlusecases/recommendations/

/home/ubuntu/dlcookbook/dlusecases/recommendations


In [3]:
PATH = './data/download/training_set/'
PATH_MERGED = './data/merged/'

# 1 Summary

This notebook creates a neural network, which predicts the user rating for a movie. The implementation can predict the rating with an error of 0.75 (MSE). <br><br>The data are part from the netflix prize <a href='http://www.netflixprize.com/'>Netflix Prize</a>:<br>
<ul>
  <li>17k movies</li>
  <li>480k users</li>
  <li>100 M ratings</li>
</ul>
<br><br>The main idea is to use embeddings for the user and movie variables. The embeddings are combined (concated) and feed through a neural network.<br><br>
<img src='./src/structure.png', style="width: 50%; height: 50%"><br><br>

Even the simple network architecture with 2 layers improves the company's recommendation engine from 2005 by 21% and improves the winning implementation from 2009 by 12%. (Note: The winning solution couldn't be implemented due to calculation duration.)

Resources:<br>
<ul>
  <li><a href='http://www.netflixprize.com/'>Netflix prize</a/></li>
  <li><a href='https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson4.ipynb'>Implementation for MoveiLens</a/></li> 
</ul>


Keywords: Deep Learning, Embeddings, Deep Collaborative Filtering, Recommendation Engine

# 2 Data Loading

Loading all CSV files.<br>
1. First merge 500 CSV files into a batch
2. Merge the batches into the final dataframe

In [4]:
def readRatings(file):
    ratings = pd.read_csv(file)
    ratings = ratings.reset_index()
    movieID = ratings.columns[2]
    ratings.columns = ['userID', 'rating', 'date']
    ratings['movieID'] = movieID.replace(':','')
    return(ratings.values.tolist())

def readMergedRatings(file):
    ratings = pd.read_csv(file)
    return(ratings.values.tolist())

In [69]:
files = listdir(PATH)
all_ratings = pd.DataFrame()
i = 0
j = 1
for file in files:
    i=i+1
    if i % 100 == 0:
        print(i)
    if i % 500 == 0:
        all_ratings.to_csv('./data/merged/' + str(j) + '.csv')
        all_ratings = pd.DataFrame()
        j = j+1
    ratings_puffer = readRatings(PATH + file)
    all_ratings = all_ratings.append(ratings_puffer)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700


In [75]:
files = listdir(PATH_MERGED)
all_ratings = pd.DataFrame()
for file in files:
    print(file)
    ratings_puffer = readMergedRatings(PATH_MERGED + file)
    all_ratings = all_ratings.append(ratings_puffer)

6.csv
19.csv
17.csv
32.csv
12.csv
21.csv
23.csv
18.csv
8.csv
2.csv
4.csv
15.csv
26.csv
14.csv
28.csv
34.csv
25.csv
27.csv
31.csv
11.csv
7.csv
10.csv
16.csv
13.csv
30.csv
33.csv
5.csv
22.csv
35.csv
1.csv
9.csv
3.csv
20.csv
24.csv
29.csv


In [77]:
all_ratings.shape

(98886378, 5)

In [78]:
all_ratings.to_csv('all_ratings.csv')

In [5]:
all_ratings = pd.read_csv('all_ratings.csv')

# 3 Data Cleaning & Featue Engineering

In [ ]:
all_ratings.columns = ['rowID','rowID','userID', 'rating', 'date', 'movieID']

Transform the UserID and MocieID to an incremental ID

In [11]:
users = all_ratings.userID.unique()
movies = all_ratings.movieID.unique()

In [12]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [14]:
all_ratings.movieID = all_ratings.movieID.apply(lambda x: movieid2idx[x])
all_ratings.userID = all_ratings.userID.apply(lambda x: userid2idx[x])

# 4 Model

In [32]:
def emb_init(shape, name=None): 
    return initializations.uniform(shape, scale=2/(shape[1]+1), name=name)


## Training/Validation Set

In [75]:
n_users = all_ratings.userID.nunique()
n_movies = all_ratings.movieID.nunique()
n_factors = 256
np.random.seed = 42

In [76]:
# The 1.1 can be changed to get a subsample
msk = np.random.rand(len(all_ratings)) < 1.1
sample_all_ratings = all_ratings[msk]

msk = np.random.rand(len(sample_all_ratings)) < 0.8
train_all_ratings = sample_all_ratings[msk]
val_all_ratings = sample_all_ratings[~msk]

In [77]:
input_user = Input((1,), dtype='int64', name='user_in')
user_in = Embedding(n_users, n_factors, input_length=1, init=emb_init)(input_user)

input_movie = Input((1,), dtype='int64', name='movie_in')
movie_in = Embedding(n_movies, n_factors, input_length=1, init=emb_init)(input_movie)

In [78]:
x = merge([user_in, movie_in], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
model = Model([input_user, input_movie], x)
model.compile(Adam(0.001), loss='mse')

In [79]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_23 (Embedding)         (None, 1, 256)        122923008   user_in[0][0]                    
____________________________________________________________________________________________________
embedding_24 (Embedding)         (None, 1, 256)        4479744     movie_in[0][0]                   
___________________________________________________________________________________________

In [80]:
model.fit([train_all_ratings.userID, train_all_ratings.movieID], train_all_ratings.rating, batch_size=2048, nb_epoch=10, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)],  validation_data=([val_all_ratings.userID, val_all_ratings.movieID], val_all_ratings.rating))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:88: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 122923008 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


79110144/|/[loss: 0.777] 100%|| 79110144/79111408 [1:34:48<00:00, 13790.92it/s]

KeyboardInterrupt: 